# Cleaning DataFrames

* Much of the data that we manage needs to be continually cleaned
* This chapter is dedicated on techniques for cleaning our data for tasks like:
  * Reporting
  * Machine Learning
* Please open a tab that shows [the API for dealing with `null` values](https://spark.apache.org/docs/2.3.0/api/scala/index.html?org/apache/spark/sql/Dataset.html#org.apache.spark.sql.DataFrameNaFunctions)